In [ ]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.tools.retriever import create_retriever_tool

In [ ]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

In [ ]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

In [ ]:
loader=WebBaseLoader("https://docs.langchain.com/oss/python/langchain/agents")
docs=loader.load()
split=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vector_store=FAISS.from_documents(split,embedding=OllamaEmbeddings(model="llama3.1:8b"))
retriever=vector_store.as_retriever()

In [ ]:
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name

In [ ]:
tools=[wiki,arxiv,retriever_tool]

In [ ]:
llm=Ollama(model="llama3.1:8b")

In [ ]:
from langchain import hub

In [ ]:
prompt=hub.pull("hwchase17/react")

In [ ]:
from langchain.agents import create_react_agent
my_agent=create_react_agent(llm,tools,prompt)

In [ ]:
# Agent executor
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=my_agent,tools=tools,verbose=True,handle_parsing_errors=True,max_iterations=5,)

In [ ]:
agent_executor.invoke({"input":"tell me about langsmith"})

In [ ]:
import streamlit as st
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun,DuckDuckGoSearchResults,DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent
from langgraph.callbacks import StreamlitCallbackHandler